# Evaluating RAG Systems

Evaluation and benchmarking are crucial in developing LLM applications. Optimizing performance for applications like RAG (Retrieval Augmented Generation) requires a robust measurement mechanism.

LlamaIndex provides essential modules to assess the quality of generated outputs and evaluate content retrieval quality. It categorizes its evaluation into two main types:

*   **Response Evaluation** : Assesses quality of Generated Outputs
*   **Retrieval Evaluation** : Assesses Retrieval quality

[Documentation
](https://docs.llamaindex.ai/en/latest/module_guides/evaluating/)

In [1]:
# !pip install llama-index

## Response Evaluation

Evaluating results from LLMs is distinct from traditional machine learning's straightforward outcomes. LlamaIndex employs evaluation modules, using a benchmark LLM like GPT-4, to gauge answer accuracy. Notably, these modules often blend query, context, and response, minimizing the need for ground-truth labels.

The evaluation modules manifest in the following categories:

*   **Faithfulness:** Assesses whether the response remains true to the retrieved contexts, ensuring there's no distortion or "hallucination."
*   **Relevancy:** Evaluates the relevance of both the retrieved context and the generated answer to the initial query.
*   **Correctness:** Determines if the generated answer aligns with the reference answer based on the query (this does require labels).

Furthermore, LlamaIndex has the capability to autonomously generate questions from your data, paving the way for an evaluation pipeline to assess the RAG application.

In [2]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

from jet.llm.ollama import initialize_ollama_settings, Ollama
initialize_ollama_settings()

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

Event: pre_start_hook
File: ipykernel_launcher.py

pre_start_hook triggered at: 2025-02-09|07:12:34


In [3]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    RetrieverEvaluator,
    generate_question_context_pairs,
)

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Response,
)

# from llama_index.llms.openai import OpenAI

import os

In [4]:
# os.environ["OPENAI_API_KEY"] = "sk-..."

#### Download Data

In [5]:
# !mkdir -p 'data/paul_graham/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#### Load Data

In [6]:
reader = SimpleDirectoryReader("/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/data/jet-resume/data/")
documents = reader.load_data()

#### Generate Question

In [7]:
gpt4 = Ollama(model="llama3.1", temperature=0.1)

dataset_generator = DatasetGenerator.from_documents(
    documents,
    llm=gpt4,
    show_progress=True,
    num_questions_per_chunk=2,
)

eval_dataset = dataset_generator.generate_dataset_from_nodes(num=1)

Parsing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/jethroestrada/Desktop/External_Projects/AI/repo-libs/llama_index/llama-index-core/llama_index/core/evaluation/dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
  0%|          | 0/1 [00:00<?, ?it/s]

Counting tokens for string: The original query is as follows: You are a Teacher/Professor. Your task is to setup                         2 questions for an upcoming                         quiz/examination. The questions should be diverse in nature                             across the document. Restrict the questions to the                                 context information provided.
We have provided an existing answer: 
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------

------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
Estimating tokens in tools...
Counting tokens for string: 
Counting tokens for string: Context information is below.
---------------------

---------------------
Given the context information and not prior knowledge.
generate only questions based on the below query.
You are a Teacher/Profess

100%|██████████| 1/1 [00:11<00:00, 11.17s/it]


)


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.1, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 406
Content: 2609

Durations:
total_duration: 9.23s
load_duration: 41.49ms
prompt_eval_duration: 697.00ms
eval_duration: 8.49s


Final tokens info:
Prompt tokens: 482
Response tokens: 98
Total tokens: 580



  0%|          | 0/5 [00:00<?, ?it/s]

Counting tokens for string: The original query is as follows: Here are two questions based on the provided context:
We have provided an existing answer: 
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------

------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
Estimating tokens in tools...
Counting tokens for string: 
Counting tokens for string: Context information is below.
---------------------

---------------------
Given the context information and not prior knowledge, answer the query.
Query: Here are two questions based on the provided context:
Answer: 
Estimating tokens in tools...
Counting tokens for string: 

Calling Ollama chat...
LLM model: llama3.1 (4096) | Tokens: 474


Model: llama3.1
Prompt: 1911
Tokens: 474
Stream: True
Generating response...
Event: call_ollama_chat
File: ipykernel_launcher.py
Log-Fi

100%|██████████| 5/5 [00:37<00:00,  7.56s/it]

".


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.1, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 223
Content: 2188

Durations:
total_duration: 4.69s
load_duration: 34.78ms
prompt_eval_duration: 429.00ms
eval_duration: 4.22s


Final tokens info:
Prompt tokens: 457
Response tokens: 50
Total tokens: 507




/Users/jethroestrada/Desktop/External_Projects/AI/repo-libs/llama_index/llama-index-core/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [8]:
eval_queries = list(eval_dataset.queries.values())

In [9]:
(eval_queries)

['Here are two questions based on the provided context:']

In [10]:
len(eval_queries)

1

To be consistent we will fix evaluation query

In [11]:
eval_query = "How did the author describe their early attempts at writing code?"

In [12]:
# Fix GPT-3.5-TURBO LLM for generating response
gpt35 = Ollama(temperature=0, model="llama3.2")

# Fix GPT-4 LLM for evaluation
gpt4 = Ollama(temperature=0, model="llama3.1")

In [13]:
# create vector index
vector_index = VectorStoreIndex.from_documents(documents, llm=gpt35)

# Query engine to generate response
query_engine = vector_index.as_query_engine()

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http

In [14]:
retriever = vector_index.as_retriever(similarity_top_k=3)
nodes = retriever.retrieve(eval_query)

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768


In [15]:
from IPython.display import display, HTML

display(HTML(f'<p style="font-size:20px">{nodes[1].get_text()}</p>'))

#### Faithfullness Evaluator

 Measures if the response from a query engine matches any source nodes. This is useful for measuring if the response was hallucinated.

In [16]:
faithfulness_evaluator = FaithfulnessEvaluator(llm=gpt4)

In [17]:
# Generate response
response_vector = query_engine.query(eval_query)

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Counting tokens for string: The original query is as follows: How did the author describe their early attempts at writing code?
We have provided an existing answer: 
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------

------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
Estimating tokens in tools...
Counting tokens for string: 
Counting tokens for string: Context information is below.
---------------------

---------------------
Given the context information and not prior knowledge, answer the query.
Query: How did the author describe their early attempts at writing code?
Answer: 
Estimating

In [18]:
eval_result = faithfulness_evaluator.evaluate_response(
    response=response_vector
)

Counting tokens for string: Please tell if a given piece of information is supported by the context.
You need to answer with either YES or NO.
Answer YES if any of the context supports the information, even if most of the context is unrelated. Some examples are provided below. 

Information: Apple pie is generally double-crusted.
Context: An apple pie is a fruit pie in which the principal filling ingredient is apples. 
Apple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard or cheddar cheese.
It is generally double-crusted, with pastry both above and below the filling; the upper crust may be solid or latticed (woven of crosswise strips).
Answer: YES
Information: Apple pies tastes bad.
Context: An apple pie is a fruit pie in which the principal filling ingredient is apples. 
Apple pie is often served with whipped cream, ice cream ('apple pie à la mode'), custard or cheddar cheese.
It is generally double-crusted, with pastry both above and below the filli

In [19]:
eval_result.passing

True

In [20]:
eval_result

EvaluationResult(query=None, contexts=['# Skills\n\n## Frontend\n\n- React\n- React Native\n- Vanilla JS/CSS\n- Expo\n- GraphQL\n- Redux\n- Gatsby\n- TypeScript\n\n## Backend\n\n- Node.js\n- Python\n\n## Databases\n\n- PostgreSQL\n- MongoDB\n\n## Platforms\n\n- Firebase\n- AWS\n- Google Cloud\n\n## Developer Tools\n\n- Photoshop\n- Jest (Unit testing)\n- Cypress (Integration testing)\n- Selenium (E2E testing)\n- Git\n- Sentry bug tracker\n- Android Studio\n- Xcode\n- Fastlane\n- Serverless\n- ChatGPT', '# Web apps\n\n## Jules Procure\n\n### Achievements\n\n- Started as the sole client side developer, built enterprise web and mobile CRM apps starting from provided mockups to production\n- JulesAI CEO was impressed and acquired ownership of existing CRM\n- Successfully integrated existing CRM with JulesAI\'s workflow to be rebranded as "Jules Procure"\n- Key features: Contact dashboard, Data builder, Task calendar, Workflow boards, Form builders, Price list generator, Automated emails ba

#### Relevency Evaluation

Measures if the response + source nodes match the query.

In [21]:
# Create RelevancyEvaluator using GPT-4 LLM
relevancy_evaluator = RelevancyEvaluator(llm=gpt4)

In [22]:
# Generate response
response_vector = query_engine.query(eval_query)

# Evaluation
eval_result = relevancy_evaluator.evaluate_response(
    query=eval_query, response=response_vector
)

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Counting tokens for string: The original query is as follows: How did the author describe their early attempts at writing code?
We have provided an existing answer: 
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------

------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
Estimating tokens in tools...
Counting tokens for string: 
Counting tokens for string: Context information is below.
---------------------

---------------------
Given the context information and not prior knowledge, answer the query.
Query: How did the author describe their early attempts at writing code?
Answer: 
Estimating

In [23]:
eval_result.query

'How did the author describe their early attempts at writing code?'

In [24]:
eval_result.response

"**Repeat**\n\nThere is no mention of the author's early attempts at writing code in this context. It appears to be a list of skills and tools used by the author, but there is no narrative or personal anecdote about their coding journey."

In [25]:
eval_result.passing

False

Relevancy evaluation with multiple source nodes.

In [26]:
# Create Query Engine with similarity_top_k=3
query_engine = vector_index.as_query_engine(similarity_top_k=3)

# Create response
response_vector = query_engine.query(eval_query)

# Evaluate with each source node
eval_source_result_full = [
    relevancy_evaluator.evaluate(
        query=eval_query,
        response=response_vector.response,
        contexts=[source_node.get_content()],
    )
    for source_node in response_vector.source_nodes
]

# Evaluation result
eval_source_result = [
    "Pass" if result.passing else "Fail" for result in eval_source_result_full
]

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Counting tokens for string: The original query is as follows: How did the author describe their early attempts at writing code?
We have provided an existing answer: 
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------

------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
Estimating tokens in tools...
Counting tokens for string: 
Counting tokens for string: Context information is below.
---------------------

---------------------
Given the context information and not prior knowledge, answer the query.
Query: How did the author describe their early attempts at writing code?
Answer: 
Estimating

In [27]:
eval_source_result

['Fail', 'Fail', 'Fail']

#### Correctness Evaluator

Evaluates the relevance and correctness of a generated answer against a reference answer.

In [28]:
correctness_evaluator = CorrectnessEvaluator(llm=gpt4)

In [29]:
query = "Can you explain the theory of relativity proposed by Albert Einstein in detail?"

reference = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

General relativity, published in 1915, extended these ideas to include the effects of gravity. According to general relativity, gravity is not a force between masses, as described by Newton's theory of gravity, but rather the result of the warping of space and time by mass and energy. Massive objects, such as planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in response to this curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet, causing it to create a depression that other objects (representing smaller masses) naturally move towards.

In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

response = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

However, general relativity, published in 1915, extended these ideas to include the effects of magnetism. According to general relativity, gravity is not a force between masses but rather the result of the warping of space and time by magnetic fields generated by massive objects. Massive objects, such as planets and stars, create magnetic fields that cause a curvature in spacetime, and smaller objects follow curved paths in response to this magnetic curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet with magnets underneath, causing it to create a depression that other objects (representing smaller masses) naturally move towards due to magnetic attraction.
"""

In [30]:
correctness_result = correctness_evaluator.evaluate(
    query=query,
    response=response,
    reference=reference,
)


Calling Ollama chat...
LLM model: llama3.1 (4096) | Tokens: 837


Model: llama3.1
Prompt: 3918
Tokens: 837
Stream: True
Generating response...
Event: call_ollama_chat
File: ipykernel_launcher.py
Log-Filename: ipykernel_launcher
4.0
The generated answer is highly relevant and mostly correct, but contains a significant mistake regarding general relativity. The reference answer clearly states that general relativity includes the effects of gravity, not magnetism. However, the generated answer correctly describes special relativity and provides some accurate information about general relativity. The inclusion of magnetism instead of gravity in the explanation of general relativity is a major error, but it does not completely undermine the overall relevance and correctness of the generated answer.


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.0, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 575
Content: 4493

Durations:
total_duration: 16.44s
load_dur

In [31]:
correctness_result

EvaluationResult(query='Can you explain the theory of relativity proposed by Albert Einstein in detail?', contexts=None, response="\nCertainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).\n\nHowever, general relativity, published in 1915, extended these ideas to include the effects of magnetism. According to general relativity, gravity is not a force between masses but rather the result of the warping of space and time by magnetic fields generated by massive objects. Massive objects, such as planets and stars, create magnetic fields that cause a curvature in spacetime, and smaller objects foll

In [32]:
correctness_result.score

4.0

In [33]:
correctness_result.passing

True

In [34]:
correctness_result.feedback

'The generated answer is highly relevant and mostly correct, but contains a significant mistake regarding general relativity. The reference answer clearly states that general relativity includes the effects of gravity, not magnetism. However, the generated answer correctly describes special relativity and provides some accurate information about general relativity. The inclusion of magnetism instead of gravity in the explanation of general relativity is a major error, but it does not completely undermine the overall relevance and correctness of the generated answer.'

## Retrieval Evaluation

Evaluates the quality of any Retriever module defined in LlamaIndex.

To assess the quality of a Retriever module in LlamaIndex, we use metrics like hit-rate and MRR. These compare retrieved results to ground-truth context for any question. For simpler evaluation dataset creation, we utilize synthetic data generation.

In [36]:
reader = SimpleDirectoryReader("/Users/jethroestrada/Desktop/External_Projects/Jet_Projects/JetScripts/data/jet-resume/data")
documents = reader.load_data()

from llama_index.core.text_splitter import SentenceSplitter

# create parser and parse document into nodes
parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100)
nodes = parser(documents)

In [37]:
vector_index = VectorStoreIndex(nodes)

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768
Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http

In [38]:
# Define the retriever
retriever = vector_index.as_retriever(similarity_top_k=2)

In [39]:
retrieved_nodes = retriever.retrieve(eval_query)

Calling OllamaEmbedding embed...
Embed model: nomic-embed-text (768)
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
Batch Tokens: 768


In [40]:
from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=2000)

**Node ID:** e8c7a22c-c9fe-46b3-84c2-cff4ef42a541<br>**Similarity:** 0.5197927857574212<br>**Text:** # Skills

## Frontend

- React
- React Native
- Vanilla JS/CSS
- Expo
- GraphQL
- Redux
- Gatsby
- TypeScript

## Backend

- Node.js
- Python

## Databases

- PostgreSQL
- MongoDB

## Platforms

- Firebase
- AWS
- Google Cloud

## Developer Tools

- Photoshop
- Jest (Unit testing)
- Cypress (Integration testing)
- Selenium (E2E testing)
- Git
- Sentry bug tracker
- Android Studio
- Xcode
- Fastlane
- Serverless
- ChatGPT<br>

**Node ID:** c1209e55-77e4-41f3-b980-82e93fbd8b06<br>**Similarity:** 0.5173908664780609<br>**Text:** # Web apps

## Jules Procure

### Achievements

- Started as the sole client side developer, built enterprise web and mobile CRM apps starting from provided mockups to production
- JulesAI CEO was impressed and acquired ownership of existing CRM
- Successfully integrated existing CRM with JulesAI's workflow to be rebranded as "Jules Procure"
- Key features: Contact dashboard, Data builder, Task calendar, Workflow boards, Form builders, Price list generator, Automated emails based on triggers, and more
- Technologies used: React, React Native, AWS Lambdas, GraphQL, Docker, Serverless, Jest

## Digital Cities PH

### Achievements

- As the lead developer, I worked on a portal that showcases the profiles of provinces and cities in the Philippines
- Developed an interactive Philippine map with clickable provinces, enabling users to access detailed descriptions and statistics for each region
- Key features: Interactive map, Search, Filtering, Fast loading, SEO-friendly
- Technologies used: React, GraphQL, React Static, Headless CMS

## ADEC Kenya, AMDATEX

### Achievements

- Built UI components from mockups using Photoshop to achieve pixel-perfect look
- Key features: Responsive, Reusable components
- Technologies used: React, jQuery, Wordpress<br>

In [41]:
qa_dataset = generate_question_context_pairs(
    nodes, llm=gpt4, num_questions_per_chunk=2
)

  0%|          | 0/7 [00:00<?, ?it/s]


Calling Ollama chat...
LLM model: llama3.1 (4096) | Tokens: 436


Model: llama3.1
Prompt: 1857
Tokens: 436
Stream: True
Generating response...
Event: call_ollama_chat
File: ipykernel_launcher.py
Log-Filename: ipykernel_launcher
Here are two questions based on the context information:

**Question 1**
What was the name of the social networking app developed by the individual during their time at 8WeekApp?

**Question 2**
Which company did the individual work for as a Web Developer from June 2012 to November 2014

 14%|█▍        | 1/7 [00:18<01:52, 18.78s/it]

?


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.0, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 288
Content: 2145

Durations:
total_duration: 16.43s
load_duration: 591.38ms
prompt_eval_duration: 10.28s
eval_duration: 5.55s


Final tokens info:
Prompt tokens: 403
Response tokens: 65
Total tokens: 468


Calling Ollama chat...
LLM model: llama3.1 (4096) | Tokens: 260


Model: llama3.1
Prompt: 1014
Tokens: 260
Stream: True
Generating response...
Event: call_ollama_chat
File: ipykernel_launcher.py
Log-Filename: ipykernel_launcher
Here are two questions based on the context information:

1. What is Jethro Reuel A. Estrada's preferred name?

(Answer can be found under "Basic Details")

2. In which university did Jethro Reuel A. Estrada earn his BS Degree in Computer Engineering?

(Answer can be found under "

 29%|██▊       | 2/7 [00:28<01:08, 13.66s/it]

...

 pitch?


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.0, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 291
Content: 1294

Durations:
total_duration: 6.80s
load_duration: 45.04ms
prompt_eval_duration: 1.72s
eval_duration: 5.04s


Final tokens info:
Prompt tokens: 185
Response tokens: 61
Total tokens: 246


Calling Ollama chat...
LLM model: llama3.1 (4096) | Tokens: 231


Model: llama3.1
Prompt: 913
Tokens: 231
Stream: True
Generating response...
Event: call_ollama_chat
File: ipykernel_launcher.py
Log-Filename: ipykernel_launcher
Here are two questions based on the context information:

**Question 1**
What programming language is used by Node.js, as mentioned in the Backend section of the skills?

A) Python
B) JavaScript
C) TypeScript
D) Java

**Question 2**
Which platform is listed among the Developer Tools, alongside Photoshop and Jest (Unit testing)?

A) Jenkins
B) Firebase
C) AWS
D

 86%|████████▌ | 6/7 [01:20<00:12, 12.16s/it]

) Git


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.0, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 367
Content: 1280

Durations:
total_duration: 9.09s
load_duration: 35.84ms
prompt_eval_duration: 1.72s
eval_duration: 7.34s


Final tokens info:
Prompt tokens: 205
Response tokens: 88
Total tokens: 293


Calling Ollama chat...
LLM model: llama3.1 (4096) | Tokens: 294


Model: llama3.1
Prompt: 1116
Tokens: 294
Stream: True
Generating response...
Event: call_ollama_chat
File: ipykernel_launcher.py
Log-Filename: ipykernel_launcher
Here are two questions based on the context information:

**Question 1**
What is the name of the website that showcases Jules Procure's portfolio?

A) https://www.julesai.com/#procure
B) https://www.digitalcitiesph.com
C) https://www.adec-kenya.com
D) https://jaba.live

**Answer: A) https://www.julesai.com/#procure**

**Question 2**
Which of the following apps has an Android link on Google Play Store?
A) Graduapp
B) EZ Myoma

100%|██████████| 7/7 [01:35<00:00, 13.58s/it]

 above


Model: llama3.1
Options: {'num_ctx': 3900, 'seed': 42, 'temperature': 0.0, 'num_keep': 0, 'num_predict': -1}
Stream: True
Response: 459
Content: 1575

Durations:
total_duration: 13.57s
load_duration: 35.86ms
prompt_eval_duration: 2.28s
eval_duration: 11.25s


Final tokens info:
Prompt tokens: 267
Response tokens: 134
Total tokens: 401



In [42]:
queries = qa_dataset.queries.values()
print(list(queries)[5])

**Question 1**


In [43]:
len(list(queries))

14

In [44]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [45]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
Query: Here are two questions based on the context information:
Metrics: {'mrr': 0.0, 'hit_rate': 0.0}



In [46]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http:/

In [47]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}
    )

    return metric_df

In [48]:
display_results("top-2 eval", eval_results)

,retrievers,hit_rate,mrr
0,top-2 eval,0.357143,0.285714
